## Mise en place du context

In [11]:
from findspark import init
init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

Le contexte est récupéré si existant et dans l'autre cas il sera créé.

In [47]:
sc = pyspark.SparkContext.getOrCreate() # ça marche pas pour moi avec sparkContext
spark = SparkSession.builder.getOrCreate() # Du coup je fais une sparkSession (ça doit revenir presque au même je pense)

## Chargement des données

In [3]:
import os.path as path
from os import makedirs

In [14]:
data_dir = "res"

In [5]:
if not path.isfile("dac.tar.gz"):
    !wget https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz

--2018-11-29 21:50:03--  https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz
Résolution de s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)… 52.218.36.34
Connexion à s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.36.34|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 4576820670 (4,3G) [application/x-gzip]
Enregistre : «dac.tar.gz»

dac.tar.gz            0%[                    ]   7,66M   929KB/s    tps 81m 14s^C


In [6]:
if not path.isdir(data_dir):
    makedirs(data_dir)

Le fichier [`dac.tar.gz`](https://s3-eu-west-1.amazonaws.com/kaggle-display-advertising-challenge-dataset/dac.tar.gz) doit se situer dans le même répertoire.

In [7]:
if not path.isfile(data_dir + "/train.txt") or not path.isfile(data_dir + "/test.txt"):
    !tar -xvzf dac.tar.gz -C $data_dir

In [16]:
df = spark.read.csv(data_dir + "/train.txt", header=False, sep="\t")

## Preparation des données

In [17]:
df2 = df.dropna()

In [18]:
print(df.count())
print(df2.count())

45840617
756554


In [27]:
df2.schema

StructType(List(StructField(_c0,StringType,true),StructField(_c1,StringType,true),StructField(_c2,StringType,true),StructField(_c3,StringType,true),StructField(_c4,StringType,true),StructField(_c5,StringType,true),StructField(_c6,StringType,true),StructField(_c7,StringType,true),StructField(_c8,StringType,true),StructField(_c9,StringType,true),StructField(_c10,StringType,true),StructField(_c11,StringType,true),StructField(_c12,StringType,true),StructField(_c13,StringType,true),StructField(_c14,StringType,true),StructField(_c15,StringType,true),StructField(_c16,StringType,true),StructField(_c17,StringType,true),StructField(_c18,StringType,true),StructField(_c19,StringType,true),StructField(_c20,StringType,true),StructField(_c21,StringType,true),StructField(_c22,StringType,true),StructField(_c23,StringType,true),StructField(_c24,StringType,true),StructField(_c25,StringType,true),StructField(_c26,StringType,true),StructField(_c27,StringType,true),StructField(_c28,StringType,true),StructFi

Convertir les colonne 0 à 13 inclu en float

In [24]:
df3 = df2.select(*(col(c).cast("float") if i < 14 else col(c) for i, c in enumerate(df2.columns)))

Renommage des colonnes par : label, f1, f2, f3,..., f39

In [46]:
def mk_newNameCol():
    res = ['label']
    for i in range(1, 40):
        res.append('f' + str(i))
    return res

new_name = mk_newNameCol()
print(new_name)

df3 = df3.select(*[col(c).alias(new_c) for c, new_c in zip(df3.columns, new_name)])
# df3.head() # Ca bug à ce moment là je sais pas pourquoi

['label', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39']


Afficher les valeurs uniques pour les attributs 14 à 39 inclu

In [40]:
df3.select(*(col(c) for c in df2.columns[14:])).distinct().show() # marche pas

AnalysisException: "cannot resolve '`_c14`' given input columns: [label];;\n'Project ['_c14, '_c15, '_c16, '_c17, '_c18, '_c19, '_c20, '_c21, '_c22, '_c23, '_c24, '_c25, '_c26, '_c27, '_c28, '_c29, '_c30, '_c31, '_c32, '_c33, '_c34, '_c35, '_c36, '_c37, ... 2 more fields]\n+- Project [label#633 AS label#638]\n   +- Project [_c0#135 AS label#633]\n      +- Project [cast(_c0#12 as float) AS _c0#135, cast(_c1#13 as float) AS _c1#136, cast(_c2#14 as float) AS _c2#137, cast(_c3#15 as float) AS _c3#138, cast(_c4#16 as float) AS _c4#139, cast(_c5#17 as float) AS _c5#140, cast(_c6#18 as float) AS _c6#141, cast(_c7#19 as float) AS _c7#142, cast(_c8#20 as float) AS _c8#143, cast(_c9#21 as float) AS _c9#144, cast(_c10#22 as float) AS _c10#145, cast(_c11#23 as float) AS _c11#146, cast(_c12#24 as float) AS _c12#147, cast(_c13#25 as float) AS _c13#148, _c14#26, _c15#27, _c16#28, _c17#29, _c18#30, _c19#31, _c20#32, _c21#33, _c22#34, _c23#35, ... 16 more fields]\n         +- Filter AtLeastNNulls(n, _c

In [27]:
df3.schema

StructType(List(StructField(_c0,FloatType,true),StructField(_c1,FloatType,true),StructField(_c2,FloatType,true),StructField(_c3,FloatType,true),StructField(_c4,FloatType,true),StructField(_c5,FloatType,true),StructField(_c6,FloatType,true),StructField(_c7,FloatType,true),StructField(_c8,FloatType,true),StructField(_c9,FloatType,true),StructField(_c10,FloatType,true),StructField(_c11,FloatType,true),StructField(_c12,FloatType,true),StructField(_c13,FloatType,true),StructField(_c14,StringType,true),StructField(_c15,StringType,true),StructField(_c16,StringType,true),StructField(_c17,StringType,true),StructField(_c18,StringType,true),StructField(_c19,StringType,true),StructField(_c20,StringType,true),StructField(_c21,StringType,true),StructField(_c22,StringType,true),StructField(_c23,StringType,true),StructField(_c24,StringType,true),StructField(_c25,StringType,true),StructField(_c26,StringType,true),StructField(_c27,StringType,true),StructField(_c28,StringType,true),StructField(_c29,Strin

## Arrêt du context

In [28]:
sc.stop()